In [3]:
!pip install --upgrade datasets[audio] transformers accelerate evaluate jiwer tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 50.2 MB/s eta 0:00:00:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 115.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 44.4 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.1
    Uninstalling tensorboard-2.15.1:
      Successfully uninstalled tensorboard-2.15.1
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.3
    Uninstalling transformers-4.42.3:
      Successfully uninstalled transformers-4.42.3
ERROR: pip's depe

In [4]:
from datasets import load_dataset, DatasetDict, concatenate_datasets
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import librosa
import librosa.display
import warnings
from transformers import AutoProcessor, AutoFeatureExtractor, AutoModelForAudioClassification
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer, pipeline
from huggingface_hub import HfApi, HfFolder, Repository
import torch
from dataclasses import dataclass
import evaluate
from typing import Any, Dict, List, Union
from datasets import Audio

warnings.filterwarnings('ignore')

2024-08-03 10:43:19.389080: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-03 10:43:19.389212: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-03 10:43:19.508270: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Load Dataset

In [5]:
lang_used=['zh-CN', 'ru-RU', 'fr-FR', 'en-US', 'de-DE']
minds=[]
for i in lang_used:
    minds_data = load_dataset("PolyAI/minds14", i, trust_remote_code=True)
    minds.append(minds_data['train'])

minds = concatenate_datasets(minds)
minds

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
    num_rows: 2754
})

In [6]:
minds = minds.train_test_split(test_size=0.4, seed = 21, stratify_by_column="intent_class")
minds_test = minds['test'].train_test_split(test_size=0.5, seed=21, stratify_by_column="intent_class")
dataset = DatasetDict({
    'train': minds['train'],
    'test': minds_test['test'],
    'valid': minds_test['train']})
# see structure
dataset

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 1652
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 551
    })
    valid: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 551
    })
})

# facebook/wav2vec2-base


In [7]:
labels = dataset["train"].features["intent_class"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [8]:
id2label[str(2)]

'app_error'

In [9]:
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

In [10]:
dataset['train'][0]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/28aa727f91fee90575c34956bab09d1716cfaf460c6afcba86a10f04a7d58b83/de-DE~CARD_ISSUES/CA3ee78582d721d7ac7c06a3dd33b035f8_0.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/28aa727f91fee90575c34956bab09d1716cfaf460c6afcba86a10f04a7d58b83/de-DE~CARD_ISSUES/CA3ee78582d721d7ac7c06a3dd33b035f8_0.wav',
  'array': array([0.00024414, 0.00048828, 0.00048828, ..., 0.00024414, 0.00024414,
         0.00024414]),
  'sampling_rate': 8000},
 'transcription': 'Hallo kurze frage ich wollte nur wissen warum meine Karte nicht funktioniert hat und ich war heute morgen einkaufen und hat nicht mehr funktioniert',
 'english_transcription': 'Hello short question I just wanted to know why my card did not work and I went shopping this morning and it no longer worked',
 'intent_class': 6,
 'lang_id': 1}

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 1652
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 551
    })
    valid: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 551
    })
})

In [12]:
dataset_run = dataset.remove_columns(["path", "transcription", "english_transcription", "lang_id"])

In [13]:
dataset_run

DatasetDict({
    train: Dataset({
        features: ['audio', 'intent_class'],
        num_rows: 1652
    })
    test: Dataset({
        features: ['audio', 'intent_class'],
        num_rows: 551
    })
    valid: Dataset({
        features: ['audio', 'intent_class'],
        num_rows: 551
    })
})

In [14]:
dataset_run['train'][0]

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/28aa727f91fee90575c34956bab09d1716cfaf460c6afcba86a10f04a7d58b83/de-DE~CARD_ISSUES/CA3ee78582d721d7ac7c06a3dd33b035f8_0.wav',
  'array': array([0.00024414, 0.00048828, 0.00048828, ..., 0.00024414, 0.00024414,
         0.00024414]),
  'sampling_rate': 8000},
 'intent_class': 6}

In [24]:
dataset_run = dataset_run.cast_column("audio", Audio(sampling_rate=16000))

In [33]:
max_duration = 25 # seconds
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000*max_duration, truncation=True
    )
    return inputs

In [34]:
encoded_minds = dataset_run.map(preprocess_function, remove_columns="audio", batched=True)
encoded_minds = encoded_minds.rename_column("intent_class", "label")

Map:   0%|          | 0/1652 [00:00<?, ? examples/s]

Map:   0%|          | 0/551 [00:00<?, ? examples/s]

Map:   0%|          | 0/551 [00:00<?, ? examples/s]

In [35]:
encoded_minds

DatasetDict({
    train: Dataset({
        features: ['label', 'input_values'],
        num_rows: 1652
    })
    test: Dataset({
        features: ['label', 'input_values'],
        num_rows: 551
    })
    valid: Dataset({
        features: ['label', 'input_values'],
        num_rows: 551
    })
})

In [36]:
accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [37]:
num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
torch.cuda.empty_cache()

In [41]:
training_args = TrainingArguments(
    output_dir="./wav2vec2_base_classification",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_minds["train"],
    eval_dataset=encoded_minds["valid"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
0,2.617100,2.597305,0.114338
1,2.643600,2.642536,0.078040
2,2.638200,2.637417,0.078040
4,2.636700,2.637399,0.074410


TrainOutput(global_step=515, training_loss=2.639084095630831, metrics={'train_runtime': 2705.7236, 'train_samples_per_second': 3.053, 'train_steps_per_second': 0.19, 'total_flos': 1.1519705127710103e+18, 'train_loss': 2.639084095630831, 'epoch': 4.987893462469733})

In [42]:
trainer.evaluate(encoded_minds["test"])

{'eval_loss': 2.6095118522644043,
 'eval_accuracy': 0.0998185117967332,
 'eval_runtime': 76.521,
 'eval_samples_per_second': 7.201,
 'eval_steps_per_second': 0.902,
 'epoch': 4.987893462469733}

In [ ]:
trainer.push_to_hub()